# PreProcessing

In [1]:
import pandas as pd
import numpy as np
import os as os
from shutil import rmtree
import cv2 as cv2
import matplotlib.pyplot as plt


Let's set some paramaters:

 * Boarder_Cut: how many pixles to cut off of each edge of the image. Some of the images have noise lines that run through the edges, so we will chip it down using this  
 * Split_Level: THe Test/Train split  
 * Noise_Threshhold: Any pixle below this value will be set to zero  
 * Base_Dir: what is the root director of where the data is stores
 * Train_Dir: where is the training data being stored  
 * Test_Dir: where is the testing Diata being stores  
 * Final_Dir: where is the final test data being stored

In [2]:
Boarder_Cut = 3
Noise_Threshhold = 5
Base_Dir = '../Data/'

Train_Dir = Base_Dir + 'Train'
Test_Dir = Base_Dir + 'Test'


IMPORT_LIST = [Base_Dir + 'train_image_data_' + x + '.parquet' for x in ['0','1','2','3']]
IMPORT_VAL_LIST = [Base_Dir + 'test_image_data_' + x + '.parquet' for x in ['0','1','2','3']]
TRAIN_CSV = Base_Dir + 'train.csv'
TEST_CSV = Base_Dir + 'test.csv'


#Df = pd.read_parquet('../Data/train_image_data_3.parquet')
#Df_csv = pd.read_csv(TRAIN_CSV)
#2.48GB in size

In [3]:
def bounding_box(img, Values = False):
    ce = 0
    re = 0
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    
    if Values:
        return rmin, rmax, cmin, cmax
    else:
        if rmin == rmax:
            rmin = 0,
            rmax = img.shape[0]
        if cmin == cmax:
            cmin = 0
            cmax = img.shape[1]
        return(img[rmin:rmax, cmin:cmax])

### Deleting and Making Directories

In [5]:
RR = pd.DataFrame(None)
RR

""


In [18]:
rmtree(Train_Dir)
rmtree(Test_Dir)
os.mkdir(Train_Dir)
os.mkdir(Test_Dir)



# Formatting the Train Group

In [19]:
Train_csv_File = pd.DataFrame(None)

for IL in IMPORT_LIST:
    print(IL)
    Df = pd.read_parquet(IL)
    

    # Extracting the raw values from the Data frame
    Df_names = pd.DataFrame({'image_id':Df['image_id'].copy(),
                           'Image_Dir': ''})
    
    Df_np = 255 - Df.iloc[:,1:].values.reshape(-1, 137,236).astype(np.uint8).copy()
    
    Df = None

    #cropping size of the image
    Df_np = Df_np[:,Boarder_Cut:-Boarder_Cut,Boarder_Cut:-Boarder_Cut]

    #Reducing noise
    Df_np[Df_np < Noise_Threshhold] = 0

    for Idx in range(Df_np.shape[0]):
        Img = bounding_box(Df_np[Idx,:,:])
        Save_Dir = Train_Dir + '/' + Df_names.iloc[Idx,0] + '.jpg'
 
        cv2.imwrite(Save_Dir, cv2.resize(Img, (160,90)))
        Df_names.iloc[Idx,1] = Save_Dir
    
    if Train_csv_File.shape[0] == 0:
        Train_csv_File = Df_names.copy()
    else:
        Train_csv_File = Train_csv_File.append(Df_names, ignore_index = True)
    
pd.merge(left = pd.read_csv(TRAIN_CSV), right = Train_csv_File, how = 'inner', on = 'image_id').to_csv(Base_Dir + 'train_extra.csv', index = False)

../Data/train_image_data_0.parquet
../Data/train_image_data_1.parquet
../Data/train_image_data_2.parquet
../Data/train_image_data_3.parquet


# Now time to format the final Test Group

In [24]:
Test_csv_File = pd.DataFrame(None)
for IL in IMPORT_VAL_LIST:
    print(IL)
    Df = pd.read_parquet(IL)
    

    # Extracting the raw values from the Data frame
    Df_names = pd.DataFrame({'image_id':Df['image_id'].copy(),
                           'Image_Dir': ''})
    
    Df_np = 255 - Df.iloc[:,1:].values.reshape(-1, 137,236).astype(np.uint8).copy()
    
    Df = None

    #cropping size of the image
    Df_np = Df_np[:,Boarder_Cut:-Boarder_Cut,Boarder_Cut:-Boarder_Cut]

    #Reducing noise
    Df_np[Df_np < Noise_Threshhold] = 0

    for Idx in range(Df_np.shape[0]):
        Img = bounding_box(Df_np[Idx,:,:])
        Save_Dir = Train_Dir + '/' + Df_names.iloc[Idx,0] + '.jpg'
 
        cv2.imwrite(Save_Dir, cv2.resize(Img, (160,90)))
        Df_names.iloc[Idx,1] = Save_Dir
    
    if Test_csv_File.shape[0] == 0:
        Test_csv_File = Df_names.copy()
    else:
        Test_csv_File = Test_csv_File.append(Df_names, ignore_index = True)
pd.merge(left = pd.read_csv(TEST_CSV), right = Test_csv_File, how = 'inner', on = 'image_id').to_csv(Base_Dir + 'test_extra.csv', index = False)

../Data/test_image_data_0.parquet
../Data/test_image_data_1.parquet
../Data/test_image_data_2.parquet
../Data/test_image_data_3.parquet


NameError: name 'Test_CSV' is not defined

In [25]:
pd.merge(left = pd.read_csv(TEST_CSV), right = Test_csv_File, how = 'inner', on = 'image_id').to_csv(Base_Dir + 'test_extra.csv', index = False)

In [26]:
Test_csv_File.shape

(12, 2)

In [27]:
pd.read_csv(TEST_CSV).shape

(36, 3)

In [ ]:
pd.merge(left = pd.read_csv(TRAIN_CSV), right = Test_csv_File, how = 'inner', on = 'image_id').to_csv(Base_Dir + 'test_extra.csv', index = False)